In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import MultiTaskLasso, Lasso
import gradio as gr
import time

rng = np.random.RandomState(42)

# Generate some 2D coefficients with sine waves with random frequency and phase
def make_plot(n_samples, n_features, n_tasks, n_relevant_features, alpha, progress=gr.Progress()):
    
    progress(0, desc="Starting...")
    time.sleep(1)
    for i in progress.tqdm(range(100)):
        time.sleep(0.1) 

    coef = np.zeros((n_tasks, n_features))
    times = np.linspace(0, 2 * np.pi, n_tasks)
    for k in range(n_relevant_features):
        coef[:, k] = np.sin((1.0 + rng.randn(1)) * times + 3 * rng.randn(1))
    
    X = rng.randn(n_samples, n_features)
    Y = np.dot(X, coef.T) + rng.randn(n_samples, n_tasks)
    
    coef_lasso_ = np.array([Lasso(alpha=0.5).fit(X, y).coef_ for y in Y.T])
    coef_multi_task_lasso_ = MultiTaskLasso(alpha=alpha).fit(X, Y).coef_
    
    fig = plt.figure(figsize=(8, 5))
    
    feature_to_plot = 0
    fig = plt.figure()
    lw = 2
    plt.plot(coef[:, feature_to_plot], color="seagreen", linewidth=lw, label="Ground truth")
    plt.plot(
        coef_lasso_[:, feature_to_plot], color="cornflowerblue", linewidth=lw, label="Lasso"
    )
    plt.plot(
        coef_multi_task_lasso_[:, feature_to_plot],
        color="gold",
        linewidth=lw,
        label="MultiTaskLasso",
    )
    #plt.legend(loc="upper center")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          ncol=3, fancybox=True, shadow=True)
    plt.axis("tight")
    plt.ylim([-1.1, 1.1])
    fig.suptitle("Lasso, MultiTaskLasso and Ground truth time series")
    return fig
   
    
model_card = f"""
## Description
Multi-task Lasso allows us to jointly fit multiple regression problems  by enforcing the selected features to be the same across tasks. This example simulates sequential measurement. Each task
is a time instant, and the relevant features, while being the same, vary in amplitude over time. Multi-task lasso imposes that features that are selected at one time point are selected
for all time points. This makes feature selection more stable than by regular Lasso.
## Model
currentmodule: sklearn.linear_model
class:`Lasso` and class: `MultiTaskLasso` are used in this example.
Plots represent Lasso, MultiTaskLasso and Ground truth time series
"""

theme = gr.themes.Glass(primary_hue=gr.themes.colors.gray, 
               secondary_hue=gr.themes.colors.sky,
               text_size=gr.themes.sizes.text_lg).set(slider_color="#b2dcf3")

with gr.Blocks(theme=theme,
               css=".gradio-container {background-color: #335960}") as demo:
    
    gr.Markdown('''
            <div>
            <h1 style='text-align: center'> Joint feature selection with multi-task Lasso </h1>
            </div>
        ''')
    gr.Markdown(model_card)
    gr.Markdown("Original example Author: Alexandre Gramfort <alexandre.gramfort@inria.fr>")
    gr.Markdown(
        "Iterative conversion by: <a href=\"https://www.deamarialeon.com\">Dea María Léon</a>"
    )
    gr.Markdown("### Please select values and click submit:")
    
    with gr.Row().style(equal_height=True):    
       n_samples = gr.Slider(50,500,value=100,step=50,label='Number of samples')
       n_features = gr.Slider(5,50,value=30,step=5,label='Features')
       n_tasks = gr.Slider(5,50,value=40,step=5,label='Tasks')
       n_relevant_features = gr.Slider(1,10,value=5,step=1,label='Relevant features')
       alpha = gr.Slider(0,10,value=1.0,step=0.5,label='Alpha Range')
            
    btn = gr.Button(value = 'Submit')

    btn.click(make_plot,inputs=[n_samples,n_features, n_tasks, n_relevant_features, alpha],outputs=[gr.Plot()])

demo.queue().launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
